In [1]:
# !pip install -U duckduckgo-search
# !pip install -qU crewai
# !pip install crewai_tools
# !pip install -qU langchain-groq
# !pip install sentence-transformers
# !pip install langchain_huggingface


In [2]:
import warnings
warnings.filterwarnings('ignore')
import os

In [3]:
from crewai import Agent, Task, Crew

In [4]:
os.environ['OPENAI_API_KEY']='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'
os.environ['OPENAI_MODEL_NAME']='llama-3.1-8b-Instant'
os.environ['OPENAI_API_BASE']='https://api.groq.com/openai/v1'

In [5]:
os.environ["SERPER_API_KEY"]='ed4d278c5de832745081465dac9fea2bae1db8ff'

In [6]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool,
  WebsiteSearchTool
)
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='/Users/mayank/Desktop/Crew_AI/Sharan_Resume_Intern_Hilti.md')


In [7]:
web_rag_tool = WebsiteSearchTool(website='https://www.cse.iitd.ac.in/index.php/2011-12-29-23-14-30/faculty',config=dict(
        llm=dict(
            provider="groq", 
            config=dict(
                model="llama-3.1-8b-Instant",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", 
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    ))

/home/mcb76/miniconda3/envs/sih1654/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Inserting batches in chromadb: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.52it/s]


In [8]:
web_rag_tool_s = WebsiteSearchTool(website='https://www.iith.ac.in/people/faculty/',config=dict(
        llm=dict(
            provider="groq", 
            config=dict(
                model="llama-3.1-8b-Instant",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", 
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    ))

Inserting batches in chromadb: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.79it/s]


In [9]:
from crewai_tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool("DuckDuckGoSearch")
def search(search_query: str):
    """Search the web for information on a given topic."""
    return DuckDuckGoSearchRun().run(search_query)

In [10]:
from crewai_tools import MDXSearchTool

semantic_search_resume = MDXSearchTool(mdx='/Users/mayank/Desktop/Crew_AI/Sharan_Resume_Intern_Hilti.md',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama-3.1-8b-Instant",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mayank/Desktop/Crew_AI/Sharan_Resume_Intern_Hilti.md'

In [15]:
drdo_resume_analyzer = Agent(
    role="DRDO-focused Resume Analyzer",
    goal="Analyze resumes for DRDO relevance, identify key defense research skills, and provide insights for DRDO career advancement",
    tools = [read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "You are a specialized Resume Analyzer with deep knowledge of DRDO's research areas, "
        "projects, and technological needs. Your expertise lies in identifying candidates with "
        "potential for defense research and development. You understand the unique requirements "
        "of DRDO positions, from entry-level scientists to senior research leads. Your analysis "
        "considers both technical skills and the innovative thinking necessary for advancing "
        "India's defense capabilities."
    )
)

In [16]:
# Task for DRDO-focused Resume Analyzer Agent: Perform DRDO-Relevant Resume Analysis
drdo_resume_analysis_task = Task(
    description=(
        "Conduct a comprehensive analysis of the provided resume with a focus on DRDO relevance. Specifically:"
        "\n1. Identify skills and experiences aligned with DRDO's key research areas which are {areas} "
        "\n2. Evaluate the candidate's potential contribution to ongoing or future DRDO projects."
        "\n3. Assess the candidate's research and innovation capabilities relevant to defense technology."
        "\n4. Analyze academic background and its relevance to DRDO's scientific domains."
        "\n5. Identify any security clearance levels or experience with classified projects."
        "\n6. Evaluate publications, patents, or other scientific contributions relevant to defense research."
        "\n7. Provide recommendations for positioning the candidate for DRDO roles or career advancement within the organization."
        
    ),
    expected_output=(
        "A detailed analytical report including:\n"
        "1. Executive Summary: Overall assessment of the candidate's fit for DRDO (2-3 sentences)\n"
        "2. DRDO Skill Alignment: Key skills matching DRDO's research areas and technological needs\n"
        "3. Project Potential: Possible DRDO projects or departments where the candidate could contribute effectively\n"
        "4. Research and Innovation Assessment: Evaluation of the candidate's potential for defense-related innovation\n"
        "5. Academic Relevance: Analysis of how the candidate's academic background supports DRDO's mission\n"
        "6. Security and Clearance: Any relevant experience with classified projects or existing clearances\n"
        "7. Scientific Contributions: Notable publications, patents, or research relevant to defense technology\n"
        "8. DRDO Career Recommendations:\n"
        "   a. Suggested DRDO roles or departments best suited for the candidate\n"
        "   b. Skill development priorities for DRDO career advancement\n"
        "   c. Potential specialized training or certifications beneficial for DRDO work\n"
        "9. Unique Insights: Any non-obvious observations about the candidate's potential in defense research\n"
        "\nThe analysis should provide actionable insights for both the candidate seeking a career in DRDO "
        "and for DRDO recruiters or managers evaluating the candidate."
    ),
    agent=drdo_resume_analyzer,
    output_file="interview_materials.md",

)

In [25]:
panel_finder_agent = Agent(
    role="Interview Panel Finder",
    goal="Identify and select a panel of 3-5 experts from IIT Hyderabad and IIT Delhi institution to evaluate a specific candidate based on the resume analysis summary.",
    tools = [web_rag_tool,web_rag_tool_s],
    verbose=True,
    backstory=(
        "You are an expert in sourcing and selecting highly qualified individuals for academic and research panels. "
        "Your task involves finding suitable experts from prestigious institutions like IIT-H to form an interview panel. "
        "The panel is intended to evaluate a specific candidate based on the summary provided by the DRDO-focused Resume Analyzer Agent. "
        "Ensure that the experts' expertise aligns with the research needs of DRDO and the candidate's profile."
    )
)


panel_finder_task = Task(
    description=(
        "Based on the resume analysis summary of the specific candidate provided by the DRDO-focused Resume Analyzer Agent, perform the following:"
        "\n1. Search for experts in relevant fields from IIT-H and IIT-D institution who can evaluate the candidate."
        "\n2. Filter potential panelists based on their expertise related to DRDO's key research areas."
        "\n3. Evaluate the panelists' previous involvement in defense research or similar high-tech projects."
        "\n4. Provide a detailed list of potential panelists including their names, affiliations, and relevant profiles."
        "\n5. Select a final panel of 3-5 experts who are most suitable for evaluating the specific candidate's qualifications and potential."
    ),
    expected_output=(
        "Final answer should have a detailed report including:\n"
        "1. Panelist List: Clearly state the Names, affiliations, and brief profiles of the selected 3-5 panelists\n"
        "2. Expert Matching: Justification for the selection based on the candidate's resume analysis summary and DRDO's research needs\n"
        "3. Recommendations: Any additional recommendations for the interview process or panel composition\n"
        "\nInclude the report in detail without fail else consequences will be harsh. The report should provide actionable insights for assembling a high-quality interview panel to evaluate the specific candidate for DRDO recruitment."
    ),
    agent=panel_finder_agent,
    output_file="interview_panel.md",
    context=[drdo_resume_analysis_task]

)


In [26]:
job_application_crew = Crew(
    agents=[drdo_resume_analyzer,panel_finder_agent],

    tasks=[drdo_resume_analysis_task,panel_finder_task],

    verbose=True
)


2024-08-27 23:50:31,275 - 8575618880 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [27]:
inputs={
'areas':'''Advanced Materials: Development of lightweight, high-strength materials for defense applications, including composites, ceramics, and special alloys.

Artificial Intelligence and Robotics: Research in AI, machine learning, and autonomous systems for military applications, including unmanned vehicles and decision-support systems.

Cybersecurity: Development of secure communication systems, encryption technologies, and protection against cyber threats.

Electronic Warfare: Designing systems for electronic intelligence, jamming, and protection against electronic attacks.

Naval Systems: Research and development of technologies for submarines, warships, sonar systems, and underwater weapons.

Radar and Communication Systems: Development of advanced radar, communication, and surveillance systems.

Life Sciences: Research in biomedical engineering, human physiology, and protective gear for soldiers operating in extreme environments.

Nuclear, Biological, and Chemical Defence: Developing protective measures and detection systems for nuclear, biological, and chemical threats.

Missiles and Weapons Systems: While well-known for missile development, DRDO also works on other weapons systems, including advanced munitions and directed energy weapons.

Combat Vehicles and Armaments: Designing and developing tanks, armored vehicles, and other ground-based weaponry.'''}

In [28]:
result = job_application_crew.kickoff(inputs=inputs)

 [2024-08-27 23:50:34][DEBUG]: == Working Agent: DRDO-focused Resume Analyzer
 [2024-08-27 23:50:34][INFO]: == Starting Task: Conduct a comprehensive analysis of the provided resume with a focus on DRDO relevance. Specifically:
1. Identify skills and experiences aligned with DRDO's key research areas which are Advanced Materials: Development of lightweight, high-strength materials for defense applications, including composites, ceramics, and special alloys.

Artificial Intelligence and Robotics: Research in AI, machine learning, and autonomous systems for military applications, including unmanned vehicles and decision-support systems.

Cybersecurity: Development of secure communication systems, encryption technologies, and protection against cyber threats.

Electronic Warfare: Designing systems for electronic intelligence, jamming, and protection against electronic attacks.

Naval Systems: Research and development of technologies for submarines, warships, sonar systems, and underwater 

In [29]:
from IPython.display import Markdown, display
display(Markdown("/Users/mayank/Desktop/Crew_AI/Interview_Panel_Selector/interview_panel.md"))



**Panelist List:**

1. **Prof. Raja Sengupta** - TiHAN Chair Professor, Transportation Wireless Communications, University of California, Berkeley
2. **Prof. Srikanth Saripalli** - TiHAN Chair Professor, Autonomous Vehicles Unmanned Systems, Texas A & M University
3. **Dr. Sri-Rajasekhar Kothapalli** - CfHE Chair Professor, Next-Generation Therapies Mechanisms of Carcinogenesis, The Pennsylvania State University
4. **Gautam Shroff** - Vice President, Chief Scientist and Head, TCS Research, Ph.D. (Rensselaer Polytechnic Institute, USA)
5. **Rajeev Shorey** - Ph.D. (IISc., Bangalore), Communication Networks,Data Analytics, Machine Learning and Artificial Intelligence

**Expert Matching:**

Based on the candidate's resume analysis summary and DRDO's research needs, the selected panelists have expertise in relevant areas such as AI and ML, Electronic Warfare, Radar and Communication Systems, and Defense Research. Prof. Raja Sengupta and Prof. Srikanth Saripalli have experience in transportation wireless communications and autonomous vehicles, which can be relevant to DRDO's research areas. Dr. Sri-Rajasekhar Kothapalli has expertise in next-generation therapies, which can be relevant to DRDO's research areas in defense medicine. Gautam Shroff and Rajeev Shorey have experience in defense research and have worked on high-tech projects, making them suitable panelists.

**Recommendations:**

1. The panelists should be briefed on the candidate's resume analysis summary and DRDO's research needs to ensure they are well-prepared for the evaluation process.
2. The panelists should be encouraged to ask questions and engage in a thorough discussion with the candidate to assess their qualifications and potential.
3. The interview process should be structured to evaluate the candidate's technical skills, research experience, and innovation potential.
4. The panelists should provide feedback on the candidate's performance and make recommendations for further evaluation or potential hiring.

**Additional Recommendations:**

1. The candidate should be provided with information on the panelists' expertise and research areas to ensure they are well-prepared for the evaluation process.
2. The candidate should be encouraged to ask questions and engage in a thorough discussion with the panelists to demonstrate their knowledge and skills.
3. The interview process should be conducted in a fair and unbiased manner, with all panelists having an equal opportunity to assess the candidate's qualifications and potential.

**Conclusion:**

The selected panelists have expertise in relevant areas and have experience in defense research, making them suitable for evaluating the candidate's qualifications and potential. The panelists should be briefed on the candidate's resume analysis summary and DRDO's research needs, and the interview process should be structured to evaluate the candidate's technical skills, research experience, and innovation potential.